### ⚠ IMPORTANT ⚠

Please ensure your Colab runtime is set to the following:

A100 GPU

Evaluation and instruction-tuning a LLM is a resource intensive process - please make sure you're using the appropriate instance.

# Model Evaluation: A Primer

Now that we've spent some time creating models with:

- Unsupervised pre-training
- Supervised fine-tuning
- Some instruction-tuning

We're ready to begin to think about how we can evaluate these models.

## Baseline Evaluation

In order to properly understand how our model is improving - we need to first start with a baseline evaluation of our model's performance.

Let's start with Mistral AI's `Mistral-7B` model.

We're going to load and compare everything in 4-bit quantization today in order to ensure we can fit the model on our Google Colab instance.

Let's start by setting up and loading our model to prepare it for evaluation.

### Load Mistral AI's Mistral-7B in 4-bit Quantization

Let's grab our dependencies, and load our model!

In [1]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

Conforming to previous notebooks - let's set up our quantization config for our model.

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

Now we have our quantization settings confirmed - let's load up our model!

In [4]:
model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

####❓ Question:

Taking a look at the [model card](https://huggingface.co/mistralai/Mistral-7B-v0.1) (and the linked resources on the card) is this an instruct-tuned model or not?

### Answer: No

### Collect and Load the Eleuther AI Evaluation Harness

Now that we have our baseline model loaded - we need to evaluate it.

For that, we'll use a tool called [Eleuther AI's LM evaluation harness](https://github.com/EleutherAI/lm-evaluation-harness). This is a specialized tool for running benchmarks on various language tasks.

Let's start by grabbing and installing it!

Why Eleuther AI's Evaluation Harness? Well - it's what powers the [Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)!

In [5]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -e .

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 30510, done.
remote: Counting objects: 100% (993/993), done.
remote: Compressing objects: 100% (466/466), done.
remote: Total 30510 (delta 650), reused 814 (delta 523), pack-reused 29517
Receiving objects: 100% (30510/30510), 22.06 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (21304/21304), done.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3

Now, we can cast our model to the desired format.

In [6]:
import lm_eval
from lm_eval.models.huggingface import HFLM
eval_model = HFLM(model, batch_size=4)

We'll set up our tasks so we can leverage them at evaluation time!

In [7]:
lm_eval.tasks.initialize_tasks()

Next, we can evaluate our base model!

>NOTE: This step will take ~30-40min. to run in full on the A100 - so ensure you set aside time to run it fully if you desire!

We're going to leverage two benchmarks today:

- [HellaSwag](https://rowanzellers.com/hellaswag/)
- [ARC Easy](https://leaderboard.allenai.org/arc_easy/submissions/get-started)
- A subset of the [MMLU benchmark](https://paperswithcode.com/dataset/mmlu), focusing only on the `machine_learning` task.

These are lightweight benchmarks used to "score" models against eachother on the OpenLM leaderboard.

We'll consider a simple average of their scores as the "overall" score of the baseline model.

You could easily extend the number of tasks considered if you wanted to more exactly emulate the Open LLM Leaderboard.

In [8]:
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["hellaswag", "arc_easy"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10042 [00:00<?, ? examples/s]

Map:   0%|          | 0/39905 [00:00<?, ? examples/s]

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 49669/49669 [17:52<00:00, 46.32it/s]


In [9]:
import pandas as pd

pd.DataFrame(results["results"])

,hellaswag,arc_easy
"acc,none",0.608743,0.798822
"acc_stderr,none",0.00487,0.008226
"acc_norm,none",0.805616,0.786195
"acc_norm_stderr,none",0.003949,0.008413
alias,hellaswag,arc_easy


In [10]:
fs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=5,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:37<00:00, 12.02it/s]


In [11]:
import pandas as pd

pd.DataFrame(fs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.401786
"acc_norm,none",0.401786
"acc_norm_stderr,none",0.046533
"acc_stderr,none",0.046533
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


### Zero-Shot MMLU

In [12]:
zs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:10<00:00, 44.75it/s]


In [13]:
import pandas as pd

pd.DataFrame(zs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.3125
"acc_norm,none",0.3125
"acc_norm_stderr,none",0.043995
"acc_stderr,none",0.043995
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


### Chain of Thought

Now let's try a Chain of Thought example!

In [14]:
cot_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_cot_zeroshot_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:num_fewshot has been set to 0 for mmlu_flan_cot_zeroshot_machine_learning in its config. Manual configuration will be ignored.
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running generate_until requests
  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation 

In [15]:
import pandas as pd

pd.DataFrame(cot_mmlu_results["results"])

,mmlu_flan_cot_zeroshot_machine_learning
alias,mmlu_flan_cot_zeroshot_machine_learning
"exact_match,get-answer",0.0
"exact_match_stderr,get-answer",0.0
